<img src='../OUTILS/bandeau_MF.png' align='right' width='100%'/>

<div class="alert alert-info alert-success">
<h3>Manipulation des données d'imagerie avec la librairie gdal </h3></div>

## <a id='TOC-TOP'></a>Contenus

Tout d'abord, il faut procéder à l'importation des librairies nécessaires à ce TP.

In [ ]:
from PIL import Image
from IPython.display import display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy
import os
os.environ['PATH'] = f"/opt/conda/envs/casablanca/bin:{os.environ['PATH']}"

Ce TP utilise en grande partie les logiciels de la librairie **"gdal"** qui signifie : *geospatial data abstraction library*. Cette librairie est extrèmement utile pour manipuler les données issues des satellites météorologiques.

Pour ce TP nous allons utiliser un fichier au format **NetCDF** issue du data store d'Eumetsat.
Ces fichiers NetCDF sont proposés selon la nomenclature suivante :
W_XX-EUMETSAT-Darmstadt,VIS+IR+HRV+IMAGERY_MSG4+SEVIRI_C_EUMG_**AAAAMMJJHHMMSS**.nc

Le fichier que nous allons manipuler est le suivant :

**`W_XX-EUMETSAT-Darmstadt,VIS+IR+HRV+IMAGERY_MSG4+SEVIRI_C_EUMG_20220905120011.nc`**

Il se trouve dans ce répertoire :
**`../../MF_DATA/MSG`**

La commande gdalinfo permet d'afficher les informations qui se trouvent dans le fichier NetCDF.

usage : `gdalinfo mon_fichier`

Pour lancer les commandes gdal dans ce jupyter notebook il faut les précéder d'un point d'exclamation :

`!gdalinfo`

In [ ]:
!gdalinfo

Maintenant, faites un gdalinfo du fichier NetCDF.

In [ ]:
!gdalinfo ../../MF_DATA/MSG/W_XX-EUMETSAT-Darmstadt,VIS+IR+HRV+IMAGERY_MSG4+SEVIRI_C_EUMG_20220905120011.nc

Il y a énormément d'informations proposées. Mais nous n'allons pas toutes les exploiter.
Nous utiliserons uniquement les informations des subdataset compris entre 37 et 47.

Si on voit un NetCDF comme une armoire, un subdatset est un tiroir de cette armoire.

Pour se simplifier la tâche, nous allons définir le chemin d'accès aux données dans une variable.

In [ ]:
input = '../../MF_DATA/MSG/W_XX-EUMETSAT-Darmstadt,VIS+IR+HRV+IMAGERY_MSG4+SEVIRI_C_EUMG_20220905120011.nc'

On peut vérifier à quoi correspond notre variable.

In [ ]:
input

!gdalinfo $input

La commande suivante permet d'afficher le contenu du subdataset d'un fichier NetCDF :
    
!gdalinfo NETCDF:"$input":**nom_du_subdataset**

***Exercice 1 : afficher les informations du canal IR 10,8 microns.***

Il est possible de rajouter des options à la commande gdalinfo.

l'option -mm permet d'afficher les valeurs mini et maxi des données que l'on consulte.

Questions : quels sont les valeurs mini et maxi du canal 10,8 microns et du canal 0,8 microns ?

Définissons la variable du dossier où seront stockés les résultats.

In [ ]:
download_dir = os.path.join(os.getcwd(), "../RESULTATS")
os.makedirs(download_dir, exist_ok=True)

In [ ]:
output = '../RESULTATS'

La commande suivante permet de générer un image à partir des données qui sont stockées dans le NetCDF :

Indiquer le nom du subdataset

In [ ]:
nom_du_subdataset = ''

In [ ]:
!gdal_translate -ot byte -scale 0 1023 0 255 NETCDF:"$input":{nom_du_subdataset} {output}/20220905_1200_IR108_0-1024.tif

Nous allons redimensionner l'image à l'aide la commande suivante :

In [ ]:
im = Image.open(output + '/20220905_1200_IR108_0-1024.tif', 'r')
image_redim = im.resize((800,800), resample=0)
image_redim.save(output + '/20220905_1200_IR108_0-1024_redim.tif')

Il est alors possible de visualiser l'image à l'aide la commande suivante :

In [ ]:
im = Image.open(output + '/20220905_1200_IR108_0-1024_redim.tif', 'r')
display(im)

L'image manque de contraste.

In [ ]:
!gdal_translate -ot byte -scale 0 871 0 255 NETCDF:"$input":{nom_du_subdataset} {output}/20220905_1200_IR108_min-max.tif

In [ ]:
im = Image.open(output + '/20220905_1200_IR108_min-max.tif', 'r')
image_redim = im.resize((800,800), resample=0)
image_redim.save(output + '/20220905_1200_IR108_min-max_redim.tif')
im = Image.open(output + '/20220905_1200_IR108_min-max_redim.tif', 'r')
display(im)

La commande suivante permet de faire une RGB :

`!gdal_merge.py -separate band1.tif band2.tif band3.tif -o resultat.tif`

***Exercice 2 : faire une RGB natural color***

Faire l'exercice dans cette cellule

Effectuons un gdalinfo d'une image créée avec gdal_translate :

In [ ]:
!gdalinfo $output/20220905_1200_IR108_min-max.tif

L'image n'est pas géoréférencée. L'étape suivante va consister à géoréférencer l'image.
Chargeons les variables de géoréférencement du satellite MSG :

In [ ]:
longitude=0
major_axis=6378169
minor_axis=6356583.8
hauteur=35785831
ulx=-5570248.832537
uly=5570248.832537
lrx=5567248.429179
lry=-5567248.429179

In [ ]:
!gdal_translate -a_srs "+proj=geos +a=$major_axis +b=$minor_axis +lon_0=$longitude +h=$hauteur +x_0=0 +y_0=0 +pm=$longitude" -a_ullr $ulx $uly $lrx $lry $output/20220905_1200_IR108_min-max.tif $output/20220905_1200_IR108_min-max_georef.tif 

Un nouveau gdalinfo donne un résultat différent

In [ ]:
!gdalinfo $output/20220905_1200_IR108_min-max_georef.tif

Redimensionnons l'image

In [ ]:
im = Image.open(output + '/20220905_1200_IR108_min-max_georef.tif', 'r')
image_redim = im.resize((800,800), resample=0)
image_redim.save(output + '/20220905_1200_IR108_redim.tif')

In [ ]:
!gdalinfo $output/20220905_1200_IR108_redim.tif

Le géoréférencement a été perdu !!

Pour le conserver, nous pouvons redimensionner l'image à l'aide la commande gdalwarp :

In [ ]:
!gdalwarp -ts 800 800 $output/20220905_1200_IR108_min-max_georef.tif $output/20220905_1200_IR108_min-max_georef_redim.tif

In [ ]:
!gdalinfo $output/20220905_1200_IR108_min-max_georef_redim.tif

Le géoréférencement est conservé.

On peut afficher l'image :

In [ ]:
im = Image.open(output + '/20220905_1200_IR108_min-max_georef_redim.tif', 'r')
display(im)

Reprojection d'une image :

In [ ]:
!gdalwarp -t_srs EPSG:4326 $output/20220905_1200_IR108_min-max_georef.tif $output/20220905_1200_IR108_min-max_georef_4326.tif

Consultons le géoréférencement :

In [ ]:
!gdalinfo $output/20220905_1200_IR108_min-max_georef_4326.tif

Affichons l'image.

In [ ]:
im = Image.open(output + '/20220905_1200_IR108_min-max_georef_4326.tif', 'r')
display(im)

Redimensionnons l'image.

In [ ]:
!gdalwarp -ts 800 800 $output/20220905_1200_IR108_min-max_georef_4326.tif $output/20220905_1200_IR108_min-max_georef_4326_redim_ts.tif

Que s'est-t-il passé ?

In [ ]:
im = Image.open(output + '/20220905_1200_IR108_min-max_georef_4326_redim_ts.tif', 'r')
display(im)

Il faut utiliser une autre méthode.

In [ ]:
!gdalwarp -tr 0.1 -0.1 $output/20220905_1200_IR108_min-max_georef_4326.tif $output/20220905_1200_IR108_min-max_georef_4326_redim_tr.tif

In [ ]:
im = Image.open(output + '/20220905_1200_IR108_min-max_georef_4326_redim_tr.tif', 'r')
display(im)

gdalwarp permet de reprojeter, mais permet aussi de découper une image.
Il faut utiliser la commande :
    
`!gdalwarp -te xmin ymin xmax ymax in.tif out.tif`

***Exercice 3 : Découper une zone qui correspond à votre pays***

In [ ]:
# Exemple France
!gdalwarp -te -14 38 18 56 -ts 1920 1080 -overwrite $output/20220905_1200_IR108_min-max_georef_4326.tif $output/20220905_1200_France.tif

In [ ]:
im = Image.open(output + '/20220905_1200_France.tif', 'r')
display(im)

***Exercice 4 : Découper une zone au format 16X9 qui correspond à votre pays***

Cette commande permet de rajouter un shapefile sur une image.
Exemple : rajout d'un contour de cote

In [ ]:
metadonnees = '../OUTILS/'

In [ ]:
!gdal_rasterize -b 1 -burn 255 -l world-administrative-boundaries $metadonnees/boundary/world-administrative-boundaries.shp $output/20220905_1200_France.tif

On peut aussi rajouter les graticules.

In [ ]:
!gdal_rasterize -b 1 -burn 255 -l 10m-graticules-5 $metadonnees/graticules/10m-graticules-5.shp $output/20220905_1200_France.tif
im = Image.open(output + '/20220905_1200_France.tif', 'r')
display(im)

**Attention !!!**

La commande gdal_rasterize écrase l'image qui est donnée en entrée.

Visualisation de l'image :
`im = Image.open(output + '/20220905_1200_France.tif', 'r')
display(im)`

La commande suivante va permettre de seuiller les données.

`!gdal_translate -scale Inputmin Inputmax Outputmin Outputmax in.tif image_transparente.tif`

***Exercice 5 : Trouver un seuillage qui ne fasse apparaitre que les nuages***

Faire l'exercice dans cette cellule

Ensuite il est possible de mettre en transparent les zones qui sont noires :
    
`!gdalwarp -srcnodata '0' -dstalpha -dstnodata '0,0' in.tif image_transparente.tif`

In [ ]:
!gdalwarp -srcnodata '0' -dstalpha -dstnodata '0,0' $output/20220905_1200_France_seuil.tif $output/20220905_1200_France_transp.tif


In [ ]:
im = Image.open(output + '/20220905_1200_France_transp.tif', 'r')
display(im)

Découpage d'un fond de carte sur son pays :
Exemple pour la France :

In [ ]:
!gdalwarp -te -14 38 18 56 -ts 1920 1080 -overwrite ../../MF_DATA/world.200408.3x21600x10800.tif $output/Fond_france-petit.tif

In [ ]:
im = Image.open(output + '/Fond_france-petit.tif', 'r')
display(im)

***Exercice 6 : découper un fond de carte qui correspond à la zone découpée précedemment***

superposition des nuages au fond de carte :

In [ ]:
img = Image.open(output + '/20220905_1200_France_transp.tif', 'r')
fond = Image.open(output + '/Fond_france-petit.tif', 'r')
fond.paste(img, (0,0), img)
fond.save(output + '/20220905_1200_France_fond.tif',"TIFF")
im = Image.open(output + '/20220905_1200_France_fond.tif', 'r')
display(im)

Ajout d'un logo sur l'image.

In [ ]:
logo1 = Image.open(metadonnees + '/Republique_Francaise_RVB.png')
petit_logo1 = logo1.resize((135,123), resample=0)
petit_logo1.save(output + '/petit_logo1.png')

In [ ]:
logo2 = Image.open(metadonnees + '/logo_mf_zoom.png')
petit_logo2 = logo2.resize((123,123), resample=0)
petit_logo2.save(output + '/petit_logo2.png')

In [ ]:
img = Image.open(output + '/20220905_1200_France_fond.tif', 'r')
petit_logo1 = Image.open(output + '/petit_logo1.png', 'r')
petit_logo2 = Image.open(output + '/petit_logo2.png', 'r')
img.paste(petit_logo1, (41,900))
img.paste(petit_logo2, (1700,900))
img.save(output + '/20220905_1200_France_fond_logo.tif')
im = Image.open(output + '/20220905_1200_France_fond_logo.tif', 'r')
display(im)


***Exercice 7 : faire une image pour son pays avec le logo de son institution***

Faire l'exercice dans cette cellule